<div style="background:#F5F7FA; height:100px; padding: 2em; font-size:14px;">
<span style="font-size:18px;color:#152935;">Want to do more?</span><span style="border: 1px solid #3d70b2;padding: 15px;float:right;margin-right:40px; color:#3d70b2; "><a href="https://ibm.co/wsnotebooks" target="_blank" style="color: #3d70b2;text-decoration: none;">Sign Up</a></span><br>
<span style="color:#5A6872;"> Try out this notebook with your free trial of IBM Watson Studio.</span>
</div>

<table style="border: none" align="left">
   <tr style="border: none">
      <th style="border: none"><font face="verdana" size="5" color="black"><b>Predicting Iris species with IBM Watson Machine Learning</b></th>
      <th style="border: none"><img src="https://github.com/pmservice/customer-satisfaction-prediction/blob/master/app/static/images/ml_icon_gray.png?raw=true" alt="Watson Machine Learning icon" height="40" width="40"></th>
  <tr style="border: none">
       <th style="border: none"><img src="https://github.com/pmservice/wml-sample-models/blob/master/spark/customer-satisfaction-prediction/images/users_banner_2-03.png?raw=true" width="600" alt="Icon"> </th>
   </tr>
</table>

This notebook contains steps from storing sample PMML model to starting scoring new data. 

Some familiarity with python is helpful. This notebook uses Python 3.

You will use a **Iris** data set, which details measurements of iris perianth. Use the details of this data set to predict iris species.

## Learning goals

The learning goals of this notebook are:

-  Working with the WML instance
-  Online deployment of PMML model
-  Scoring of deployed model


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Work with service instance](#instance) 
3.	[Persist, deploy and score in an IBM Cloud](#scoring)
4.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Setup

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a [Watson Machine Learning Service](https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/) instance required for model storing and deployment (a free plan is offered).
-   Install [watson-machine-learning-client](https://pypi.python.org/pypi?%3Aaction=search&term=watson-machine-learning-client&submit=search) (not required on DSx).

<a id="instance"></a>
## 2. Working with the instance

In this section you will learn how to get details of your instance.

Install python API client from pypi.

In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [1]:
!pip install watson-machine-learning-client --upgrade

**Action**: Put credentials from your instance of Watson Machine Learning service in IBM Cloud here.

In [2]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "access_key": "***",
  "username": "***",
  "password": "***",
  "instance_id": "***"
}

In [3]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [4]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

Details about your Watson Machine Learning service are available in `instance_details`.

In [5]:
print(wml_client.instance_details)

{'metadata': {'created_at': '2018-01-30T10:39:51.938Z', 'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e', 'modified_at': '2018-01-30T14:16:00.601Z', 'guid': 'fd1db635-c9f3-4d83-afc7-32efec367e5e'}, 'entity': {'region': 'us-south', 'account': {'id': '8dba8395f5b8dc6fcab49d43161ab9f1', 'type': 'TRIAL', 'name': "Wojciech Sobala's Account"}, 'deployments': {'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/deployments'}, 'source': 'Bluemix', 'status': 'Active', 'plan_id': '0f2a3c2c-456b-40f3-9b19-726d2740b11c', 'published_models': {'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models'}, 'organization_guid': 'a7b3ef43-5dc2-41fc-8fa7-29b0846e5807', 'plan': 'standard', 'space_guid': 'a6b628f9-3f64-45c0-bf38-739876b63fec', 'usage': {'deployment_count': {'current': 1, 'limit': 1000}, 'expiration_date': '2018-02-01T00:00:00.000Z', 'comput

Getting stored models on WML instance

In [6]:
wml_client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  -----------------  -----
GUID                                  NAME                            CREATED                   FRAMEWORK          TYPE
51cdbdea-28ea-488d-9edd-dab2a8c44208  Sentiment Prediction            2018-01-30T14:15:06.339Z  mllib-2.0          model
b7e9fc69-f0db-444f-89b4-fe7aa5e8a4f1  Handwritten Digits Recognition  2018-01-30T14:15:24.106Z  scikit-learn-0.17  model
------------------------------------  ------------------------------  ------------------------  -----------------  -----


Geting list of deployed models.

In [7]:
wml_client.deployments.list()

------------------------------------  ------------------  ------  ------------------------  -----------------
GUID                                  NAME                TYPE    CREATED                   FRAMEWORK
06faa2cd-e12b-4595-b96d-555d9fae6363  Digits recognition  online  2018-01-30T14:15:59.342Z  scikit-learn-0.17
------------------------------------  ------------------  ------  ------------------------  -----------------


<a id="scoring"></a>
## 3. Persist, deploy and score model in a Cloud

In this section you will learn how to create online deployment and to score a new data record by using the python API client for Watson Machine Learning service.
For more information, see the [python API client documentation](http://wml-api-pyclient.mybluemix.net/).

### 3.1: Persist model

**Action**: Download sample PMML model from git project using wget.

In [8]:
import wget, os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

Store downloaded file in Watson Machine Learning repository.

In [9]:
props_pmml = {wml_client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              wml_client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "pmml", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "4.2"}

model_details = wml_client.repository.store_model(filename, props_pmml)
wml_client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  -----------------  -----
GUID                                  NAME                            CREATED                   FRAMEWORK          TYPE
51cdbdea-28ea-488d-9edd-dab2a8c44208  Sentiment Prediction            2018-01-30T14:15:06.339Z  mllib-2.0          model
b7e9fc69-f0db-444f-89b4-fe7aa5e8a4f1  Handwritten Digits Recognition  2018-01-30T14:15:24.106Z  scikit-learn-0.17  model
fae8e498-136e-4c35-81bd-a9a69a50b1c7  CHAID PMML model for Iris data  2018-01-30T14:17:19.921Z  pmml-4.2           model
------------------------------------  ------------------------------  ------------------------  -----------------  -----


**Note:** You can see that model is successfully stored in Watson Machine Learning Service.

### 3.2: Create online deployment

You can use commands bellow to create online deployment for stored model.

In [10]:
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Iris species prediction')

wml_client.deployments.list()

------------------------------------  -----------------------  ------  ------------------------  -----------------
GUID                                  NAME                     TYPE    CREATED                   FRAMEWORK
06faa2cd-e12b-4595-b96d-555d9fae6363  Digits recognition       online  2018-01-30T14:15:59.342Z  scikit-learn-0.17
56f3501f-844c-4cb7-9bbc-02131da8f6bf  Iris species prediction  online  2018-01-30T14:17:22.742Z  pmml-4.2
------------------------------------  -----------------------  ------  ------------------------  -----------------


### 3.3: Scoring

You can check online scoring endpoint by sending sample record using python API client.

In [11]:
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/fd1db635-c9f3-4d83-afc7-32efec367e5e/published_models/fae8e498-136e-4c35-81bd-a9a69a50b1c7/deployments/56f3501f-844c-4cb7-9bbc-02131da8f6bf/online


You can see that scoring url is available for sending scoring requests.

In [12]:
scoring_data = {'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

{'fields': ['PredictedValue', 'Confidence'], 'values': [['setosa', 1.0]]}


As we can see this is Iris Setosa flower.

<a id="summary"></a>
## 4. Summary and next steps     

 You successfully completed this notebook! You learned how to use Watson Machine Learning for PMML model deployment and scoring. Check out our _[Online Documentation](https://console.ng.bluemix.net/docs/services/PredictiveModeling/index.html?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

Wojciech Sobala, Data Scientist in IBM.

Copyright © 2018 IBM. This notebook and its source code are released under the terms of the MIT License.